In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
walk = pd.read_csv("walkability.csv")
happy = pd. read_csv("happy_data.csv")

In [3]:
walk["CBSA_Name"] = walk["CBSA_Name"].replace("(-.*,)", ",", regex = True)

In [4]:
walk = walk[["CBSA_Name", "NatWalkInd", "TotPop"]].dropna()

walkPops = walk[["CBSA_Name", "TotPop"]].groupby(by = "CBSA_Name").sum()
walkPops = walkPops.rename(columns = {"TotPop":"AggPop"})
walk = pd.merge(walk, walkPops, on = "CBSA_Name", how = "left")
walk["walkContrib"] = walk["NatWalkInd"]*(walk["TotPop"]/walk["AggPop"])
walk = walk[["CBSA_Name", "walkContrib"]].groupby(by="CBSA_Name").sum().rename(columns = {"walkContrib" : "AvgWalk"})
masterList = pd.merge(walk, happy, left_on = "CBSA_Name", right_on= "City")
masterList

,AvgWalk,Overall Rank,City,Total Score,Emotional & Physical Well-Being,Income & Employment,Community & Environment
0,7.250872,168,"Akron, OH",39.65,165,168,156
1,10.870719,128,"Albuquerque, NM",49.04,92,98,174
2,9.991200,156,"Amarillo, TX",44.51,164,122,74
3,8.856140,38,"Anchorage, AK",59.96,42,84,57
4,8.513197,97,"Atlanta, GA",52.36,91,101,116
...,...,...,...,...,...,...,...
94,10.431548,61,"Tampa, FL",57.85,79,29,34
95,9.669269,172,"Toledo, OH",38.29,175,142,160
96,9.890744,134,"Tucson, AZ",47.83,134,93,137
97,8.945005,151,"Tulsa, OK",45.11,162,110,77


In [5]:
y = masterList["Total Score"]
x = sm.add_constant(masterList["AvgWalk"])

In [6]:
est = sm.OLS(y, x)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:            Total Score   R-squared:                       0.231
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     29.06
Date:                Thu, 08 Jun 2023   Prob (F-statistic):           4.93e-07
Time:                        22:53:18   Log-Likelihood:                -338.36
No. Observations:                  99   AIC:                             680.7
Df Residuals:                      97   BIC:                             685.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         30.1346      3.988      7.556      0.0